In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from astropy.time import Time

def load_latest(ptype):
    
    lists = glob.glob(f'../target_lists/Ariel_MCS_{ptype}*.csv')
    
    dates = []
    
    for l in lists:
        t = Time(f'{l[-14:-4]}T00:00:00.000')
        dates.append(t.jd)
    
    return lists[np.where(dates==np.max(dates))[0][0]]

def load_list(ptype, name):
    
    if name == 'latest':
        df = pd.read_csv(load_latest(ptype))
    else:
        df = pd.read_csv(f'../target_lists/Ariel_MCS_{ptype}_{date}.csv')
        
    return df

def filter_tl(df, filter_dic):
    
    for para in list(filter_dic.keys()):
        df = df.loc[(df[para]>=filter_dic[para]['min']) & (df[para]<=filter_dic[para]['max'])]
    
    return df.reset_index(drop=True)

def plot_parameter_versus_observations(known, tpcs, parameter, obs_col, colour_col, cmap, xlim, ylim, save_plot=False,output_name='placeholder_name'):
    
    fs=18
    fig=plt.figure(figsize=(10,6))
    fig.patch.set_facecolor('w')
    plt.scatter(known[parameter], known[obs_col],c=known[colour_col],
           cmap=cmap,marker='o',edgecolor='k',lw=0.5,s=120,label='Known')
    plt.scatter(tpcs[parameter], tpcs[obs_col],c=tpcs[colour_col],
           cmap=cmap,marker='*',edgecolor='k',lw=0.5,s=170,label='TPCs')
    cbar=plt.colorbar()
    cbar.set_label(label=colour_col,size=fs)
    cbar.ax.tick_params(labelsize=fs-4)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xlabel(parameter,fontsize=fs)
    plt.ylabel(f'Ariel {obs_col}',fontsize=fs)
    plt.xticks(fontsize=fs-3)
    plt.yticks(fontsize=fs-3)
    
    plt.legend(loc='center', bbox_to_anchor=(0.5, 1.05),ncol=2)
    if save_plot:
        plt.savefig(f"plots/{output_name}.png",dpi=200,bbox_inches='tight')


known = load_list('Known', 'latest')
tpcs = load_list('TPCs', 'latest')

parameter = 'Planet Radius [Re]'
obs_col = 'Tier 1 Observations'
colour_col = 'Planet Temperature [K]'
save_plot = True
cmap = 'plasma'
xlim = [0.1,25]
ylim = [0,10.5]
output_name = 'radius_vs_tier1_observations'

plot_parameter_versus_observations(known, tpcs, parameter, obs_col, colour_col, \
                                   cmap, xlim, ylim, save_plot, output_name)

In [ ]:
#The above plot is rather useless. Let's focus on a specfic type of planets...

dic = {}
dic['Planet Radius [Re]'] = {'min': 0., 'max': 2.}
dic['Planet Temperature [K]'] = {'min': 1800., 'max': 4000.}

known = load_list('Known', 'latest')
tpcs = load_list('TPCs', 'latest')

known_usp = filter_tl(known, dic)
tpcs_usp = filter_tl(tpcs, dic)

parameter = 'Planet Radius [Re]'
obs_col = 'Tier 1 Observations'
colour_col = 'Planet Temperature [K]'
save_plot = True
cmap = 'plasma'
xlim = [0.5,2.02]
ylim = [0,10.5]
output_name = 'usp_radius_vs_tier1_observations'

plot_parameter_versus_observations(known_usp, tpcs_usp, parameter, obs_col, colour_col, \
                                   cmap, xlim, ylim, save_plot, output_name)

In [ ]:
def plot_hist(known,tpcs,tier,plot_target=True):
    
    targets = [1000,500,100]
    max_obs = [10,20,5] #rough recommendations
    
    bins=np.arange(-0.5,max_obs[tier-1]+2,1)
    
    known_values, known_base = np.histogram(known[f'Tier {tier} Observations'], bins=bins)
    known_cumulative = np.cumsum(known_values)
    
    tpcs_values, tpcs_base = np.histogram(tpcs[f'Tier {tier} Observations'], bins=bins)
    tpcs_cumulative = np.cumsum(tpcs_values)
    
    fs=18
    plt.figure(figsize = (10,6))
    ax1=plt.subplot(111)
    plt.plot(known_base[:-1]+0.5, known_cumulative, c ='r', lw = 2.,label='Known')
    plt.plot(tpcs_base[:-1]+0.5, tpcs_cumulative, c ='#1E90FF', lw = 2.,label='TPCs')
    plt.plot(known_base[:-1]+0.5, known_cumulative+tpcs_cumulative, c ='#FF8C00', lw = 2.,label='Known + TPCs')
    
    if plot_target:
        plt.plot([0,max_obs[tier-1]],[targets[tier-1],targets[tier-1]],ls = ':', c = 'k',lw=2,label='Ariel Goal')
    
    plt.xlim([0,max_obs[tier-1]])
    plt.ylim([0,np.max(known_cumulative+tpcs_cumulative)+100])
    plt.legend(fontsize=fs-3,ncol=2,loc='upper left')
    plt.xticks(fontsize=fs-2)
    plt.yticks(fontsize=fs-2)
    plt.xlabel('Number of Observations',fontsize=fs)
    plt.ylabel('Number of Planets',fontsize=fs)
    plt.title(f'Potential Tier {tier} Targets',fontsize=fs-2)
    
    ax2=ax1.twinx()
    plt.ylim([0,np.max(known_cumulative+tpcs_cumulative)+100])
    plt.yticks(fontsize=fs-2)
    
    plt.savefig(f'plots/tier{tier}_planets_hist.png',dpi=200,bbox_inches='tight')

known = load_list('Known', 'latest')
tpcs = load_list('TPCs', 'latest')

plot_hist(known,tpcs,1,plot_target=True)
plot_hist(known,tpcs,2,plot_target=True)
plot_hist(known,tpcs,3,plot_target=True)